In [4]:
import math
from io import BytesIO
from PIL import Image
import requests
from matplotlib import pyplot as plt
import pandas as pd
from itertools import product

TILE_SIZE = 256
URL = "https://tile.openstreetmap.org/{z}/{x}/{y}.png".format

def point_to_pixels(lon, lat, zoom):
    """convert gps coordinates to web mercator"""
    r = math.pow(2, zoom) * TILE_SIZE
    lat = math.radians(lat)

    x = int((lon + 180.0) / 360.0 * r)
    y = int((1.0 - math.log(math.tan(lat) + (1.0 / math.cos(lat))) / math.pi) / 2.0 * r)

    return x, y

df = pd.read_csv("stations.csv",index_col='id')
ids = [28079024, 28079038, 28079008, 28079047, 28079050, 28079048, 28079048, 28079099, 28079026, 28079006, 28079022, 28079001, 28079015 ]

#ids = [28079024, 28079038, 28079008, 28079040, 28079036, 28079018, 28079011, 28079004, 28079016, 28079039, 28079027]
df = df.loc[ids]
print(df)

top, bot = df.lat.max()+0.025, df.lat.min()-0.025 #add some margins to crop
lef, rgt = df.lon.min()-0.025, df.lon.max()+0.025 # add some margins to crop

zoom = 13
x0, y0 = point_to_pixels(lef, top, zoom)
x1, y1 = point_to_pixels(rgt, bot, zoom)

x0_tile, y0_tile = int(x0 / TILE_SIZE), int(y0 / TILE_SIZE)
x1_tile, y1_tile = math.ceil(x1 / TILE_SIZE), math.ceil(y1 / TILE_SIZE)

assert (x1_tile - x0_tile) * (y1_tile - y0_tile) < 50, "That's too many tiles!"

# full size image we'll add tiles to
img = Image.new('RGB', (
    (x1_tile - x0_tile) * TILE_SIZE,
    (y1_tile - y0_tile) * TILE_SIZE))

# loop through every tile inside our bounded box
for x_tile, y_tile in product(range(x0_tile, x1_tile), range(y0_tile, y1_tile)):
    with requests.get(URL(x=x_tile, y=y_tile, z=zoom)) as resp:
        tile_img = Image.open(BytesIO(resp.content))

    # add each tile to the full size image
    img.paste(
        im=tile_img,
        box=((x_tile - x0_tile) * TILE_SIZE, (y_tile - y0_tile) * TILE_SIZE))

#x, y = x0_tile * TILE_SIZE, y0_tile * TILE_SIZE

#img = img.crop((
#    int(x - x0),  # left
#    int(y - y0),  # top
#    int(x - x1),  # right
#    int(y - y1))) # bottom

dx = abs(x1-x0)
dy = abs(y1-y0)
sx=16
sy=16
if(dx>dy):
    sx=16
    sy=(16*dy)/dx
else:
    sy=16
    sx=(16*dx)/dy

fig, ax = plt.subplots(1,1,figsize=(sx,sy))
ax.scatter(df.lon, df.lat, alpha=0.5, c='blue', s=2500)
ax.imshow(img, extent=(lef, rgt, bot, top))
ax.set_ylim(bot, top)
ax.set_xlim(lef, rgt)

for i in ids:
    print(df.loc[i]['name'])
    ax.annotate(df.loc[i]['name'], (df.loc[i].lon, df.loc[i].lat))

plt.imshow(img)
plt.show()




KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Int64Index([28079099, 28079026, 28079006, 28079022, 28079001, 28079015], dtype='int64', name='id'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"